<a href="https://colab.research.google.com/github/eltonalenca90/YT-video-metadata-recovery/blob/main/Extra%C3%A7%C3%A3o_de_metadados_de_videos_do_youtube_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Extração de metadados de videos do youtube**

**Criado por** : Elton Alencar e Victoria Guimaraes({enda;vsg}@icomp.ufam.edu.br)

**Descrição:** Código desenvolvido para busca de videos no Youtube e extração de metadados e caption, utilizando a API do YOUTUBE

**última atualização**: 10/27/2021 

Referências: 
https://developers.google.com/youtube/v3/docs/search?hl=pt-br
https://pypi.org/project/youtube-transcript-api/ 


# Imports *

In [1]:
#instalando biblioteca
!pip install youtube_transcript_api
!pip install gspread

#imports
%reload_ext google.colab.data_table
from google.colab import data_table
from apiclient.discovery import build
import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time
from youtube_transcript_api import YouTubeTranscriptApi


#Funções e Métodos *

In [8]:
def buscar_canais_por_nome(nome, youtube):
  request = youtube.search().list(q=nome, part='snippet', type='channel', maxResults=10)
  response = request.execute()
  return response

def criar_nova_planilha(nome, numLinhas, numColumn, shClient):
  #cria uma pagina para cada uma nova página na planilha
  shClient.add_worksheet(nome, numLinhas, numColumn)

def cria_varias_planilhas(listNomes, numLinhas, numColumn, shClient):
  #cria uma tabela para cada canal
  for i in range(len(listNomes)):
    shClient.add_worksheet('{}'.format(listNomes[i]), 100, 10)
  print('criou')  

#acessando os ids do videos 
def get_videosID(videos):
  ids_videos = [] #estrutura de dados que armazenara os ids dos 50 videos
  for i in range(len(videos['items'])): #enquanto i estiver no intervalo de [0, 'quantidade de elementos da lista videos']
    ids_videos.append(videos['items'][i]['id']['videoId']) #adiciona o id do video (que está na posição i da lista videos[items])
  return ids_videos #retorna como resposta a lista ids_video

#acessando os 'channelTitle' 
def get_channelsTitle(res):
  channels = []
  for i in range(len(res['items'])):
    channels.append(res['items'][i]['snippet']['channelTitle'])
  return channels 

#acessando os titles 
def get_titles(res):
  titles = []
  for i in range(len(res['items'])):
    titles.append(res['items'][i]['snippet']['title'])
  return titles    

#acessando os 'publishTime' 
def get_publishDate(res):
  timestamps = []
  for i in range(len(res['items'])):
    timestamps.append(res['items'][i]['snippet']['publishTime'])
  return timestamps

#acessando os 'description' 
def get_descriptions(res):
  descriptions = []
  for i in range(len(res['items'])):
    descriptions.append(res['items'][i]['snippet']['description'])
  return descriptions

#função que adiciona os valores desejados numa lista 
def get_statistic_key(videos_statistic, keyDesejada):
  listaValues = []
  for i in range(len(videos_statistic)):
    if keyDesejada in videos_statistic[i]['statistics']: #verifica se o video atual possui os valores de key
      listaValues.append(videos_statistic[i]['statistics'][keyDesejada])
    else:
      listaValues.append('null')
  return listaValues

#busca pelos videos desejados (termo de consulta) em cada canal
#extrai caracteristicas desejadas (title, desc, numviews...)
#salva num csv
def busca_por_termo(termo_de_busca, apartirDe):

  arquivo = open("{}.txt".format(termo_de_busca), "a")

  df_all_data_returned = pd.DataFrame()

  lista_de_videos = youtube.search().list(q=termo_de_busca, part='snippet', type='video', maxResults=50, publishedAfter=apartirDe).execute() 
  if (len(lista_de_videos['items']) != 0):
    #extração de dados desejados
    lista_videosID = get_videosID(lista_de_videos)
    timestamps_v = get_publishDate(lista_de_videos)
    titles_videos = get_titles(lista_de_videos)
    channelsName = get_channelsTitle(lista_de_videos)  
    descriptionList = get_descriptions(lista_de_videos)

    # Call the videos.list method to retrieve statistics details for each video.
    video_response = youtube.videos().list(id=lista_videosID, part='statistics').execute()
    videos_statistic = video_response.get('items', [])

    # Select a likesCount_list
    likesCount_list = get_statistic_key(videos_statistic, 'likeCount')
    # Select a likesCount_list
    viewsCount_list = get_statistic_key(videos_statistic, 'viewCount')
    # Select a commentCount_list
    commentCount_list = get_statistic_key(videos_statistic, 'commentCount')
    # Select a dislikeCount_list
    dislikeCount_list = get_statistic_key(videos_statistic, 'dislikeCount')

    df_search = {
        'id': lista_videosID,
        'title': titles_videos,
        'channel': channelsName,
        'date_p': timestamps_v,
        'description': descriptionList,
        'views': likesCount_list,
        'likes': viewsCount_list,
        'dislikes': dislikeCount_list,
        'comments': commentCount_list
      }

    columns = [
        'id',
        'title',
        'channel',
        'date_p',
        'description',
        'views',
        'likes',
        'dislikes',
        'comments'
    ]

    df_all_data_returned = pd.DataFrame(df_search, columns=columns)
    print(df_all_data_returned.shape)

  return df_all_data_returned  

# Módulo 1: 

* Extração de dados 
* Organização de dados 
* Permanencia de dados  

##Conectando com API 
* youtube-v3


### Youtube v3

In [ ]:
#TODO : substituir com o valor da chave da API gerada por vc:
API_KEY = "AIzaSyBZXYtvA7******************4Q4" #chave de autenticação do projeto, para acesso dos dados 

In [4]:
#Construct a Resource object for interacting with an API. The serviceName and version are the names from the Discovery service.
youtube = build('youtube', 'v3', developerKey=API_KEY)
type(youtube)

googleapiclient.discovery.Resource

## Video Recovery - Busca de Videos 
* busca de videos por canal 
* extração de dados desejados
* busca com diferentes TERMOS DE CONSULTA


### Pesquisa por diferentes Termos de Busca

#### Novas Buscas

Processo:

* 1. busca de dados, pelo termo
* 2. coloca no banco de dados (concatena) 
* 3. remove duplicados 
* 4. get captions para os ids novos
* 5. coloca os novos captions no df criado no passo 2
* 6.  

In [13]:
termo_de_busca = 'extração de dados do youtube'
publicaçõesApartir = '2021-05-01T00:00:00Z'
df_dados_retornados = busca_por_termo(termo_de_busca, publicaçõesApartir)

(50, 9)


In [14]:
#vizualizando dados buscados: 
df_dados_retornados.head(10)

,id,title,channel,date_p,description,views,likes,dislikes,comments
0,0spvJseeYHU,InstaLead® 4.0 - INOVAMOS | Líder em Extração ...,Instalead Software,2021-09-08T02:19:39Z,Compre diretamente no site: https://instalead....,71,2904,2,11
1,WHBp211rifg,Informações que não podem faltar na tabela de ...,sejaphd,2021-10-26T23:17:21Z,,0,6,0,0
2,6Fgo3V90woQ,Big Data CNPJ - Extração de Milhares de Dados ...,Metodo Extração de Leads,2021-08-11T19:44:38Z,Mais de 44milhões de dados de empresas de todo...,4,268,1,1
3,zWpJGsYVn_I,Extração de dados da web para o excel de forma...,Anderson Veloso,2021-07-27T18:45:08Z,Demonstro como importar dados da web para o ex...,26,484,0,6
4,YBtMx6JBlwo,Aula 2 - Reconhecimento de padrões nos PDF&#39...,Auditando Tudo,2021-09-16T22:30:18Z,Aprenda a Apurar Indicadores Financeiros Autom...,4,16,0,2
5,qQn2jY2nuAA,Como expandir seu negócio em um mundo cada vez...,Gestão do Amanhã,2021-10-27T00:39:04Z,Hoje você vai entender porque ser ágil é um im...,161,1125,0,1
6,osa0kxdiFm8,Do THIS to craft a compelling title people WIL...,TubeBuddy,2021-10-26T15:14:46Z,How to craft a compelling title people WILL cl...,251,2364,2,174
7,uLL006Xk4b4,Extração de dados oficiais sobre Nascidos Vivo...,Gestão em Ato,2021-07-05T22:19:44Z,Vídeo tutorial de navegação no site do DATASUS...,6,156,0,0
8,xxOGZa5HZG8,SAP + Power BI,Grupo da Produtividade,2021-10-26T22:00:19Z,Vamos apresentar nesta aula os principais conc...,14,121,0,2
9,3vJGJ5i1ilg,Google l Solicitações oficiais de informações ...,Google Brasil,2021-10-26T19:30:01Z,Como o Google atende às solicitações oficiais ...,150,1292,3,13


##### Salvando em um csv

In [15]:
from google.colab import files
nomeDoArquivo = 'all_videos_metadados.csv'
df_dados_retornados.to_csv(r'all_videos_metadados.csv', index = False)
files.download(nomeDoArquivo) #download do csv

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#####Get Captions:
Busca por caption [legenda gerada automática] para cada um dos videos retornados no df_retornado

source: https://pypi.org/project/youtube-transcript-api/ 

In [16]:
allIds = df_dados_retornados.id.values
null_lista = []
allCaptions = []

while True:
  for i in range(len(allIds)):
    try:
      #acessa o caption do video e add na lista de allCaptions
      caption = YouTubeTranscriptApi.get_transcript(allIds[i], ['pt'])
      allCaptions.append(caption)
      print(str(i) + ': ' + allIds[i])
      time.sleep(6)
    except:
      #caso não tenha caption
      null_lista.append(i) #salvo o index do id que não tem caption
      allCaptions.append(['NULL']) #add null lista allCaption 
      i+=1 #incrementa i atual
      continue #continua task
  break

0: 0spvJseeYHU
2: 6Fgo3V90woQ
3: zWpJGsYVn_I
4: YBtMx6JBlwo
5: qQn2jY2nuAA
6: osa0kxdiFm8
7: uLL006Xk4b4
8: xxOGZa5HZG8
9: 3vJGJ5i1ilg
10: K30TpP1JM5k
11: Et9FYt6ZZB8
12: t7tlMdyZ6HM
14: xXl_hFWMkaE
15: US4YC9DvHJE
16: ujFHKTAIi0k
17: JbOROfflpr0
18: 5CHLH9LU1Uc
19: rMTf6MnP8Ps
20: OGudoUMqNpw
21: dRcA_2Xw2FU
23: ti-njetItug
24: kOqYHNlCmbg
25: v2S-WH3JCBo
26: Dc4lqcR66Ls
27: nfAqwiH_CFg
28: BPgXu2wjOcs
29: IlUz8klip5I
30: TeeoYeMAXkY
31: Phs_OD1MN6M
32: uAByChUOpdM
34: djV9Hkk7DFU
35: pC3wmBu1_kY
36: WHLBNptrokE
37: Nj57VD3xKdk
38: jorHmZKROBA
39: 66asE8bZ9lA
40: djQIeGMaHt8
41: -uMAnkxkS5w
42: 5adFNM9yASU
43: Y4yQaWS1MMA
44: OyIKhJ0xg4c
47: 7zmRSbztWeU
48: SE8SNOF1KLY
49: SLwWrM6wRbo


In [23]:
%unload_ext google.colab.data_table
df_metadados_com_caption = df_dados_retornados
df_metadados_com_caption['captions']=allCaptions
df_metadados_com_caption.head(5)

,id,title,channel,date_p,description,views,likes,dislikes,comments,captions
0,0spvJseeYHU,InstaLead® 4.0 - INOVAMOS | Líder em Extração ...,Instalead Software,2021-09-08T02:19:39Z,Compre diretamente no site: https://instalead....,71,2904,2,11,"[{'text': 'e', 'start': 0.54, 'duration': 2.97..."
1,WHBp211rifg,Informações que não podem faltar na tabela de ...,sejaphd,2021-10-26T23:17:21Z,,0,6,0,0,[NULL]
2,6Fgo3V90woQ,Big Data CNPJ - Extração de Milhares de Dados ...,Metodo Extração de Leads,2021-08-11T19:44:38Z,Mais de 44milhões de dados de empresas de todo...,4,268,1,1,"[{'text': 'o', 'start': 0.0, 'duration': 4.14}..."
3,zWpJGsYVn_I,Extração de dados da web para o excel de forma...,Anderson Veloso,2021-07-27T18:45:08Z,Demonstro como importar dados da web para o ex...,26,484,0,6,"[{'text': 'Oi tudo bem', 'start': 0.0, 'durati..."
4,YBtMx6JBlwo,Aula 2 - Reconhecimento de padrões nos PDF&#39...,Auditando Tudo,2021-09-16T22:30:18Z,Aprenda a Apurar Indicadores Financeiros Autom...,4,16,0,2,"[{'text': 'e vamos continuar', 'start': 1.23, ..."


In [38]:
#estrutura de um item do caption
df_metadados_com_caption.captions.values[0][1]

{'duration': 5.25,
 'start': 1.1,
 'text': 'fala galera então estamos aqui novamente'}

In [39]:
#extraindo somente o texto caption[legenda] do video 0  
c = df_metadados_com_caption.captions.values
for i in c[0]:
    print(i['text'])

e
fala galera então estamos aqui novamente
apresentando para vocês a estabilidade
4.0 É isso mesmo as paredes aí tá de
cara nova com novas funcionalidades e se
tornando aí a líder do mercado
deles se tratando sobre infrações né
tamo junto como sempre o começo aí
respeitando a todos os nossos
concorrentes então tem funcionalidades
novas e Aquino Itália e não tem o nossos
concorrentes Mas como eu digo sempre né
Livre Mercado Pode ser que eles copiem
nossas funcionalidades eu só queria dar
um recado importante falar para vocês
que a está linda ela só é comercializada
aqui no nosso canal do YouTube que tem
um link que leva com a nossa página quem
está beach.com.br ou e no nosso parceiro
de vendas que é o Frank que o site dele
hesa puro
mais.com.br que também comercializa
instale Então são dois canais é que
realizam as vendas é infelizmente as
versões anteriores elas foram criadas e
a gente tem uma diferença é muito
engraçado porque assim a ferramenta
criada ela custa mais cara do que a
fer

####Salvando novo csv


In [22]:
from google.colab import files
nomeDoArquivo = 'all_dados_com_caption.csv'
df_metadados_com_caption.to_csv(r'all_dados_com_caption.csv', index = False)
files.download(nomeDoArquivo) #download do csv

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>